In [1]:
import data_loader
from data_loader import DataFolder

2781
597


C:\Users\alexg\anaconda3\envs\dsp_project\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [4]:
def test_model(testloader, model):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = (correct/total)*100
    return accuracy

In [16]:
dataloaders = {'train': data_loader.train_loader, 'test': data_loader.test_loader}
dataset_sizes = {'train': len(data_loader.train_dataset), 'test': len(data_loader.test_dataset)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [17]:
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
k_folds = 5
num_epochs = 25
batch_size = 64
best_result = 0.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
  
# Set fixed random number seed
torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_ids, test_ids) in enumerate(kfold.split(data_loader.train_dataset)):
    print('---- Fold {}/{} ----'.format(fold+1, k_folds))
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    train_loader = DataLoader(dataset=data_loader.train_dataset, batch_size=batch_size,sampler=train_subsampler)
    test_loader = DataLoader(dataset=data_loader.train_dataset, batch_size=batch_size,sampler = test_subsampler)
    
    dataloaders = {'train': data_loader.train_loader, 'test': data_loader.test_loader}
    dataset_sizes = {'train': len(data_loader.train_dataset), 'test': len(data_loader.test_dataset)}
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    if best_acc > best_result:
        best_result=best_acc
        best_model=model
print('Best val Acc after k-folds: {:4f}'.format(best_result))        
model_fit=best_model


---- Fold 1/5 ----
Epoch 1/25
----------
train Loss: 0.4328 Acc: 0.7943
test Loss: 0.2377 Acc: 0.9045

Epoch 2/25
----------
train Loss: 0.2124 Acc: 0.9123
test Loss: 0.2041 Acc: 0.9263

Epoch 3/25
----------
train Loss: 0.1812 Acc: 0.9216
test Loss: 0.1437 Acc: 0.9380

Epoch 4/25
----------
train Loss: 0.1400 Acc: 0.9450
test Loss: 0.0957 Acc: 0.9615

Epoch 5/25
----------
train Loss: 0.1476 Acc: 0.9428
test Loss: 0.1502 Acc: 0.9447

Epoch 6/25
----------
train Loss: 0.1341 Acc: 0.9468
test Loss: 0.1709 Acc: 0.9330

Epoch 7/25
----------
train Loss: 0.1475 Acc: 0.9443
test Loss: 0.1448 Acc: 0.9464

Epoch 8/25
----------
train Loss: 0.1136 Acc: 0.9579
test Loss: 0.1614 Acc: 0.9430

Epoch 9/25
----------
train Loss: 0.1262 Acc: 0.9500
test Loss: 0.1641 Acc: 0.9313

Epoch 10/25
----------
train Loss: 0.1095 Acc: 0.9583
test Loss: 0.1035 Acc: 0.9564

Epoch 11/25
----------
train Loss: 0.1299 Acc: 0.9518
test Loss: 0.1436 Acc: 0.9447

Epoch 12/25
----------
train Loss: 0.1102 Acc: 0.9579
t

train Loss: 0.1580 Acc: 0.9374
test Loss: 0.1355 Acc: 0.9531

Epoch 21/25
----------
train Loss: 0.1577 Acc: 0.9443
test Loss: 0.0946 Acc: 0.9548

Epoch 22/25
----------
train Loss: 0.1593 Acc: 0.9392
test Loss: 0.1445 Acc: 0.9464

Epoch 23/25
----------
train Loss: 0.1256 Acc: 0.9515
test Loss: 0.1303 Acc: 0.9514

Epoch 24/25
----------
train Loss: 0.1371 Acc: 0.9500
test Loss: 0.1065 Acc: 0.9615

Epoch 25/25
----------
train Loss: 0.1512 Acc: 0.9443
test Loss: 0.1471 Acc: 0.9464

Training complete in 43m 30s
Best val Acc: 0.961474
---- Fold 5/5 ----
Epoch 1/25
----------
train Loss: 0.1557 Acc: 0.9367
test Loss: 0.0967 Acc: 0.9581

Epoch 2/25
----------
train Loss: 0.1372 Acc: 0.9479
test Loss: 0.1053 Acc: 0.9548

Epoch 3/25
----------
train Loss: 0.1544 Acc: 0.9432
test Loss: 0.1473 Acc: 0.9447

Epoch 4/25
----------
train Loss: 0.1421 Acc: 0.9439
test Loss: 0.1149 Acc: 0.9615

Epoch 5/25
----------
train Loss: 0.1538 Acc: 0.9389
test Loss: 0.1514 Acc: 0.9380

Epoch 6/25
----------


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [19]:
test_model(data_loader.test_loader, model_fit)

94.9748743718593

In [ ]:
torch.save(model_ft, './models/resnet.pth